In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with HBase configuration
spark = SparkSession.builder \
    .appName("WriteToMySQL222") \
    .config("spark.jars", "/tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar,/tmp/mysql-connector-java-8.0.29.jar,/tmp/hbase-spark-1.1.0-SNAPSHOT.jar") \
    .config("spark.extraListeners", "datahub.spark.DatahubSparkListener")\
    .config("spark.datahub.rest.server", "http://10.208.164.167:8080")\
    .config("spark.datahub.metadata.dataset.materialize", "true")\
    .config("spark.datahub.metadata.dataset.experimental_include_schema_metadata", "true")\
    .getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")
# cau hinh chuan hbase tren mang
    # .config("spark.jars", 
#"/tmp/hbase-spark-1.0.1-SNAPSHOT_spark331_hbase2415.jar,\
# /tmp/hbase-spark-protocol-shaded-1.0.1-SNAPSHOT_spark331_hbase2415.jar,
# /tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar,\
# /tmp/hbase-shaded-mapreduce-2.4.15.jar,
# /hbase-site.xml.jar,
# /tmp/scala-parser-combinators_2.12-2.1.1.jar,
# /tmp/htrace-core4-4.2.0-incubating.jar") \


# backup
    # .config("spark.hadoop.hbase.zookeeper.quorum", "localhost") \
    # .config("spark.hadoop.hbase.zookeeper.property.clientPort", "2181") \
    # .config("spark.hadoop.hbase.master", "localhost:16000") \
    # .config("spark.executor.extraClassPath", "/usr/local/hbase/lib/")\
    # .config("spark.driver.extraClassPath", "/usr/local/hbase/lib/")\
# /hbase-site.xml.jar, ?????
#     /tmp/hbase-client-2.4.14.jar,/tmp/hbase-server-2.4.14.jar,/tmp/hbase-mapreduce-2.4.15.jar
#     /tmp/hbase-spark3-protocol-shaded-1.0.0.7.2.17.0-334.jar
#    /tmp/phoenix5-spark3-6.0.0.7.2.17.0-334.jar,/tmp/phoenix-hbase-compat-2.4.1-5.1.3.jar,/tmp/scala-parser-combinators_2.12-2.1.1.jar,\
#    /usr/local/phoenix/phoenix-client-hbase-2.4-5.1.3.jar,/tmp/phoenix-core-5.1.3.jar,\
    # /tmp/hbase-client-2.4.14.jar,/tmp/hbase-server-2.4.14.jar,/tmp/hbase-mapreduce-2.4.15.jar, \

24/12/04 16:53:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = [('2','Telangana','Chhetri'),
  ('3','Sikkim','Bhutia'),
  ('4','Hyderabad','Shabbir'),
  ('5','Kerala','Vijayan'),
  ('6','Mizoram','Lalpekhlua')
  ]
columns = ["id","city","name"]
goalsDF = spark.createDataFrame(data=data, schema = columns)
goalsDF.show()

goalsDF.write.mode("overwrite").parquet("hdfs://hadoop:9000/tmp/test")


24/12/04 16:53:29 DEBUG OpenLineageRunEventBuilder: Visiting query plan Optional[== Parsed Logical Plan ==
SaveIntoDataSourceCommand org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider@427db629, Map(url -> *********(redacted), driver -> com.mysql.cj.jdbc.Driver, dbtable -> person, user -> my_user, password -> *********(redacted)), Overwrite
   +- LogicalRDD [ID#0L, Name#1, Age#2L], false

== Analyzed Logical Plan ==
SaveIntoDataSourceCommand org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider@427db629, Map(url -> *********(redacted), driver -> com.mysql.cj.jdbc.Driver, dbtable -> person, user -> my_user, password -> *********(redacted)), Overwrite
   +- LogicalRDD [ID#0L, Name#1, Age#2L], false

== Optimized Logical Plan ==
SaveIntoDataSourceCommand org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider@427db629, Map(url -> *********(redacted), driver -> com.mysql.cj.jdbc.Driver, dbtable -> person, user -> my_user, password -> *********(r

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


24/12/04 16:53:30 INFO RemovePathPatternUtils: Removing path pattern from dataset name person
24/12/04 16:53:30 DEBUG HdfsPathDataset: path: person
24/12/04 16:53:30 DEBUG RemovePathPatternUtils: Transformed path is person
24/12/04 16:53:30 DEBUG SparkSQLExecutionContext: Posting event for end 0: {"eventTime":"2024-12-04T09:53:29.552Z","producer":"https://github.com/OpenLineage/OpenLineage/tree/1.16.0/integration/spark","schemaURL":"https://openlineage.io/spec/2-0-2/OpenLineage.json#/$defs/RunEvent","eventType":"COMPLETE","run":{"runId":"01939116-ca5d-7b24-b571-a9915a2ac31a","facets":{"parent":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/1.16.0/integration/spark","_schemaURL":"https://openlineage.io/spec/facets/1-0-1/ParentRunFacet.json#/$defs/ParentRunFacet","run":{"runId":"01939116-ca59-7d64-89c0-0f8942dab124"},"job":{"namespace":"default","name":"write_to_my_sql222"}},"spark_properties":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/1.16.0/integration

In [4]:
#write to hbase
goalsDF.write\
.format("org.apache.hadoop.hbase.spark")\
.option("hbase.columns.mapping","id STRING :key, name STRING cf:name, city STRING cf:city")\
.option("hbase.namespace", "default")\
.option("hbase.table", "testspark")\
.option("hbase.spark.use.hbasecontext", False)\
.save() 

24/12/04 16:53:31 DEBUG OpenLineageRunEventBuilder: Physical plan executed [{"class":"org.apache.spark.sql.execution.command.DataWritingCommandExec","num-children":1,"cmd":[{"class":"org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand","num-children":1,"outputPath":null,"staticPartitions":null,"ifPartitionNotExists":false,"partitionColumns":[],"fileFormat":null,"options":null,"query":0,"mode":null,"outputColumnNames":"[id, city, name]"},{"class":"org.apache.spark.sql.execution.LogicalRDD","num-children":0,"output":[[{"class":"org.apache.spark.sql.catalyst.expressions.AttributeReference","num-children":0,"name":"id","dataType":"string","nullable":true,"metadata":{},"exprId":{"product-class":"org.apache.spark.sql.catalyst.expressions.ExprId","id":16,"jvmId":"37d93951-5992-41e9-81e6-ec80e03a096c"},"qualifier":[]}],[{"class":"org.apache.spark.sql.catalyst.expressions.AttributeReference","num-children":0,"name":"city","dataType":"string","nullable":true,"metadata":{

[Stage 5:==========================================>              (15 + 5) / 20]

24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb0103 closed
24/12/04 16:53:32 INFO ClientCnxn: EventThread shut down for session: 0x100035c67eb0103
24/12/04 16:53:32 INFO ClientCnxn: EventThread shut down for session: 0x100035c67eb010c
24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb010c closed
24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb010a closed
24/12/04 16:53:32 INFO ClientCnxn: EventThread shut down for session: 0x100035c67eb010a
24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb00fe closed
24/12/04 16:53:32 INFO ClientCnxn: EventThread shut down for session: 0x100035c67eb00fe
24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb00fc closed
24/12/04 16:53:32 INFO ClientCnxn: EventThread shut down for session: 0x100035c67eb00fc
24/12/04 16:53:32 INFO ClientCnxn: EventThread shut down for session: 0x100035c67eb0102
24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb0102 closed
24/12/04 16:53:32 INFO ZooKeeper: Session: 0x100035c67eb0106 clo

24/12/04 16:53:32 DEBUG ClientCnxn: Reading reply sessionid:0x100035c67eb0101, packet:: clientPath:null serverPath:null finished:false header:: 4,-11  replyHeader:: 4,619,0  request:: null response:: null
24/12/04 16:53:32 DEBUG ClientCnxn: Disconnecting client for session: 0x100035c67eb0101
24/12/04 16:53:32 DEBUG ClientCnxn: An exception was thrown while closing send thread for session 0x100035c67eb0101 : Unable to read additional data from server sessionid 0x100035c67eb0101, likely server has closed socket
24/12/04 16:53:32 DEBUG OpenLineageRunEventBuilder: Physical plan executed [{"class":"org.apache.spark.sql.execution.command.ExecutedCommandExec","num-children":0,"cmd":[{"class":"org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand","num-children":0,"query":[{"class":"org.apache.spark.sql.execution.LogicalRDD","num-children":0,"output":[[{"class":"org.apache.spark.sql.catalyst.expressions.AttributeReference","num-children":0,"name":"id","dataType":"string","nullab

In [5]:
spark.read.parquet("hdfs://hadoop:9000/tmp/test").show()

24/12/04 16:53:32 DEBUG OpenLineageRunEventBuilder: Visiting query plan Optional[== Parsed Logical Plan ==
SaveIntoDataSourceCommand org.apache.hadoop.hbase.spark.DefaultSource@7718aa26, Map(hbase.columns.mapping -> id STRING :key, name STRING cf:name, city STRING cf:city, hbase.namespace -> default, hbase.table -> testspark, hbase.spark.use.hbasecontext -> false), ErrorIfExists
   +- LogicalRDD [id#16, city#17, name#18], false

== Analyzed Logical Plan ==
SaveIntoDataSourceCommand org.apache.hadoop.hbase.spark.DefaultSource@7718aa26, Map(hbase.columns.mapping -> id STRING :key, name STRING cf:name, city STRING cf:city, hbase.namespace -> default, hbase.table -> testspark, hbase.spark.use.hbasecontext -> false), ErrorIfExists
   +- LogicalRDD [id#16, city#17, name#18], false

== Optimized Logical Plan ==
SaveIntoDataSourceCommand org.apache.hadoop.hbase.spark.DefaultSource@7718aa26, Map(hbase.columns.mapping -> id STRING :key, name STRING cf:name, city STRING cf:city, hbase.namespace -